In [4]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

from src.Quantum_Optimizer_FGS import Quantum_Optimizer_FGS

In [5]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('seq2.lp')

N = target_mdl.get_num_vars()
print(N)

#print(target_mdl.prettyprint())

* file is empty: seq2.lp - exiting
0


In [3]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -42.100
   optimal solution (binary): [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


In [4]:
qubo = QuadraticProgramToQubo().convert(target_mdl)

#print(qubo.prettyprint())

In [5]:
num_qubits = N
print(f'number of system qubits: {num_qubits}')

number of system qubits: 22


In [6]:
reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 66


In [7]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [8]:
def objective(x):
    objval = qubo.objective.evaluate(x)
    return objval

In [9]:
print(f'reference value: {cplex_optval:.3f}')
print('-'*80)

QuantumOptimizer = Quantum_Optimizer_FGS()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler)

theta_initial = np.hstack([np.full(num_qubits, np.pi/2), np.zeros(num_qubits*ansatz.reps)])

result = QuantumOptimizer.run(
    objective = objective, 
    op_trans = lambda x: x,
    qc_params_init = theta_initial,
    max_epoch = 4, 
    random_update = True, 
    intermediate_result_display = True,
    refval = cplex_optval
)

reference value: -42.100
--------------------------------------------------------------------------------
epoch = 0 	 step = 0 	 best val = 429.800
epoch = 0 	 step = 1 	 best val = 148.600
epoch = 0 	 step = 2 	 best val = 126.700
epoch = 0 	 step = 3 	 best val = 126.700
epoch = 0 	 step = 4 	 best val = -2.100
epoch = 0 	 step = 5 	 best val = -2.100
epoch = 0 	 step = 6 	 best val = -3.300
epoch = 0 	 step = 7 	 best val = -4.600
epoch = 0 	 step = 8 	 best val = -24.100
epoch = 0 	 step = 9 	 best val = -24.100
epoch = 0 	 step = 10 	 best val = -24.100
epoch = 0 	 step = 11 	 best val = -24.800
epoch = 0 	 step = 12 	 best val = -25.100
epoch = 0 	 step = 13 	 best val = -25.100
epoch = 0 	 step = 14 	 best val = -25.100
epoch = 0 	 step = 15 	 best val = -25.100
epoch = 0 	 step = 16 	 best val = -25.100
epoch = 0 	 step = 17 	 best val = -25.100
epoch = 0 	 step = 18 	 best val = -25.100
epoch = 0 	 step = 19 	 best val = -25.100
epoch = 0 	 step = 20 	 best val = -25.100
epoch

In [10]:
x = QuantumOptimizer.best_x
objval = qubo.objective.evaluate(x)

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[Quantum Result]
   objective:   -42.100
   solution:    [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0]
   is feasible: True
--------------------------------------------------------------------------------------------------------------
